# Study – Glass – Stretcher: bissection

In [ ]:
import pandas as pd
import tensorflow as tf

from milp import codify_network
from teste import get_minimal_explanation

In [ ]:
# For type annotations
import numpy as np

In [ ]:
dataset_name = 'glass'

training_data = pd.read_csv(f'datasets/{dataset_name}/train.csv')
testing_data = pd.read_csv(f'datasets/{dataset_name}/test.csv')

dataframe = pd.concat([training_data, testing_data])

keras_model = tf.keras.models.load_model(f'datasets/{dataset_name}/model_2layers_{dataset_name}.h5')

data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

In [ ]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)

### Printing What the Model Predicted

_Aka_ printing the network output.

In [ ]:
i = 0
print('i =', i)
network_input = data[i, :-1]
network_input = tf.reshape(tf.constant(network_input), [1, -1])
network_output = keras_model.predict(tf.constant(network_input))[0]
network_output = tf.argmax(network_output)

predictions = keras_model.predict(tf.constant(network_input))[0, 0]

print(f'Predictions: (ndarray[ndarray[{type(predictions)}]])', predictions)
classification: np.int64 = network_output.numpy()
print(f'Network output: ({type(classification)})', classification)

### Printing the Minimal Explanation

Minimal eplanations only indicates which inputs are relevant to get to a conclusion.

**Note:** The explanation happens _after_ the keras_model make its predictions.

In [ ]:
mdl_aux = mp_model.clone()

minimal_explanation = get_minimal_explanation(mdl_aux, network_input, network_output, n_classes, 'fischetti', output_bounds)
minimal_explanation

### OUTDATED!

> The keras_model predicted $C_1$ because:
>
> $x_0 = 2.967691214515491$,
>
> $x_3 = -1.408120229258977$,
>
> $x_5 = -0.790702170757714$, 
>
> $x_6 = 4.24127975754059$, 
>
> $x_7 = -0.3615292659832898$ and 
>
> $x_8 = -0.6037614142464092$.

### Trying to Improve the Explanation

Given a minimal explanation, can we improve it?

Constraints of type $x = c$ are equivalent to $x \le c \land x \ge c$.

Therefore, we need to substitute each $x = c$ constraint by the $x \le c$ and $x \ge c$ constraints.

Then, we try stretching the interval by substituting $x \le c$ by $x \le c + \Delta x$ and see if our prediction changes. If the prediction stays the same, then we substitue and try stretching it again. If the prediction changes, then this new interval isn't valid and we don't substitute. We found the upper bound of the interval, i.e. $x \le c$.

Then we try to stretch the interval to fin the lower bound. Analogously, We try substituting $c \ge x$ by ????/


We will end up with a pair of constraints the looks like $c - k_l \cdot \Delta{x} \le x$ and $x \le c + k_u \cdot \Delta{x}$, i.e. this pair represents $c - k_l \cdot \Delta{x} \le x \le c + k_u \cdot \Delta{x}$.

### Setting Up

In [ ]:
import docplex

In [ ]:
minimal_model = mdl_aux
testing_model = minimal_model.clone()

#### Quick Sratch

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')

for constraint in linear_constraints:
	testing_model.remove_constraint(constraint)
	testing_model.add_constraint(constraint.lhs <= constraint.rhs.clone(), 'input LE')
	testing_model.add_constraint(constraint.lhs >= constraint.rhs.clone(), 'input GE')

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

In [ ]:
def get_x_ge_value_constraints(linear_constraints):
	# x >= c
	ge_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.GE:
			ge_constraints.append(xp)
	return ge_constraints

In [ ]:
def get_x_le_value_constraints(linear_constraints):
	# x <= c
	le_constraints = []
	for xp in linear_constraints:
		if xp.sense == docplex.mp.constants.ComparisonType.LE:
			le_constraints.append(xp)
	return le_constraints

##### The Algorithm

In [ ]:
epsilon = 0.01

variables = testing_model.find_matching_vars('x')

print('Initial constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

for constraint in linear_constraints:
	variable = constraint.lhs
	testing_model.solve()
	while testing_model.solution is None:
		# delta = variable.up
		if constraint.sense == docplex.mp.constants.ComparisonType.LE:
			# x <= c
			# Try x <= c + e
			pass

print()
print('Final constraints:')
print(get_x_ge_value_constraints(linear_constraints))
print(get_x_le_value_constraints(linear_constraints))

In [ ]:
# constraints = []
# for c in constraints:
# 	pass
m = docplex.mp.model.Model()
c = None
x = c.lhs

e0 = c.rhs.constant
e1 = x.ub
c.rhs = e1
m.solve()
if m.solution is None:  # i.e. it works
    pass

**TODO:** Rewrite pair of expression of type $x \le c$ and $x \ge c$ to $x = c$:

In [ ]:
linear_constraints = testing_model.find_matching_linear_constraints('input')
linear_constraints

Note that `x_6` is actually equal to `4.24127975754059`

In [ ]:
number_of_inputs = len(dataframe.columns.drop('target'))
for i in range(number_of_inputs):
	constraints_of_x_i = filter(lambda x: x.lhs.name == f'x_{i}', linear_constraints)
	constraints = [c for c in constraints_of_x_i]

	if len(constraints) == 2:
		if constraints[0].rhs.constant == constraints[1].rhs.constant:
			testing_model.remove_constraints(constraints)
			testing_model.add_constraint(constraints[0].lhs == constraints[0].rhs, 'input')

In [ ]:
improved_explanation = testing_model.find_matching_linear_constraints('input')
improved_explanation

### Pretty Printing the Explanation

In [ ]:
def get_variable_index(variable: docplex.mp.dvar.Var) -> int:
	index = variable.name.split('_')[1]
	return int(index)

In [ ]:
def print_explanation(explanation: list[docplex.mp.constr.LinearConstraint]):
	for e in explanation:
		variable = e.lhs
		index = get_variable_index(variable)
		feature_name = dataframe.columns[index]
		print(feature_name, e.sense.operator_symbol, e.rhs)

In [ ]:
print_explanation(improved_explanation)

## Comparing with Anchor

In [ ]:
from anchor import utils

### Loading the Dataset

In [ ]:
d = utils.load_csv_dataset(
	data=f'datasets/{dataset_name}/test.csv',
	target_idx=-1,
	feature_names=['RI','Na','Mg','Al','Si','K','Ca','Ba','Fe','target'],
	# categorical_features=None,
	# features_to_use=None,
	# feature_transformations=None,
	# discretize=False,
	# balance=False,
	# fill_na='-1',
	# filter_fn=None,
	skip_first=True
)

### Explainer

In [ ]:
from anchor import anchor_tabular

In [ ]:
explainer = anchor_tabular.AnchorTabularExplainer(
	d.class_names,
	d.feature_names,
	d.train,
	d.categorical_names
)

In [ ]:
predict_fn = lambda x: tf.argmax(keras_model.predict(x)[0]).numpy().reshape(1)

In [ ]:
for a in d.train:
	a == data[i]

In [ ]:
exp = explainer.explain_instance(data[i, :-1], predict_fn)

In [ ]:
exp.names()

In [ ]:
print_explanation(improved_explanation)

In [ ]:
exp.precision()

In [ ]:
exp.coverage()

In [ ]:
# explainer.explain_instance(
# 	data_row,
# 	classifier_fn,
# 	threshold=0.95,
# 	delta=0.1,
# 	tau=0.15,
# 	batch_size=100,
# 	max_anchor_size=None,
# 	desired_label=None,
# 	**kwargs)